## Phase 2 — Structured LLM-as-Judge


- Judge provides structured evaluation with multiple criteria

In [1]:
import os
import json
from openai import OpenAI
from typing import Dict, List


client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

MODEL_ANSWER = "gpt-4o-mini"   # responder
MODEL_JUDGE  = "gpt-4.1"        # judge


In [2]:
def generate_answer(question: str) -> str:
    
    """LLM generates an answer to a question"""
    message = client.chat.completions.create(
        model=MODEL_ANSWER,
        max_tokens=1000,
        messages=[
            {
                "role": "user",
                "content": f"Answer this question: {question}"
            }
        ]
    )
    return message.choices[0].message.content

In [3]:
def judge_answer(question: str, answer: str) -> dict:
    
    """LLM judge with a structured output format"""
    judge_prompt = f"""You are an expert judge. Assess the following answer.

Question: {question}

Answer: {answer}

Evaluate on these criteria (score 1-10 for each):
1. Accuracy: Factual correctness
2. Completeness: Covers all aspects of the question
3. Clarity: Easy to understand
4. Relevance: Directly addresses the question
5. Conciseness: Not overly verbose

Respond ONLY with valid JSON in this exact format:
{{
    "scores": {{
        "accuracy": <number>,
        "completeness": <number>,
        "clarity": <number>,
        "relevance": <number>,
        "conciseness": <number>
    }},
    "overall_score": <number>,
    "strengths": ["strength1", "strength2"],
    "weaknesses": ["weakness1", "weakness2"],
    "recommendation": "accept" or "reject" or "revise",
    "feedback": "brief explanation"
}}
"""

    message = client.chat.completions.create(
        model=MODEL_JUDGE,
        max_tokens=1000,
        messages=[
            {"role": "user", "content": judge_prompt}
        ]
    )

    response_text = message.choices[0].message.content

    # Extract JSON from response
    try:
        if "```json" in response_text:
            response_text = response_text.split("```json")[1].split("```")[0]
        elif "```" in response_text:
            response_text = response_text.split("```")[1].split("```")[0]
        
        return json.loads(response_text.strip())
    except json.JSONDecodeError:
        return {
            "error": "Failed to parse JSON",
            "raw_response": response_text
        }

In [4]:
def print_evaluation(evaluation: Dict):
    
    """Pretty print the evaluation"""
    if "error" in evaluation:
        print(f"Error: {evaluation['error']}")
        print(f"Raw response: {evaluation['raw_response']}")
        return
    
    print("\n📊 EVALUATION SCORES:")
    for criterion, score in evaluation['scores'].items():
        bar = "█" * score + "░" * (10 - score)
        print(f"  {criterion.capitalize():<15} [{bar}] {score}/10")
    
    print(f"\n  Overall Score: {evaluation['overall_score']}/10")
    print(f"\n✅ Strengths:")
    for strength in evaluation['strengths']:
        print(f"  • {strength}")
    
    print(f"\n⚠️  Weaknesses:")
    for weakness in evaluation['weaknesses']:
        print(f"  • {weakness}")
    
    print(f"\n💡 Recommendation: {evaluation['recommendation'].upper()}")
    print(f"\n📝 Feedback: {evaluation['feedback']}")

In [5]:
questions = [
        "What is machine learning?",
        "Explain quantum entanglement"
    ]
    
print("=" * 70)
print("PHASE 2: STRUCTURED LLM-AS-JUDGE")
print("=" * 70)
    
for i, question in enumerate(questions, 1):
    print(f"\n\n{'='*70}")
    print(f"EXAMPLE {i}")
    print(f"{'='*70}")
    print(f"\n❓ Question: {question}\n")
        
    # Generate answer
    print("🤖 Generating answer...")
    answer = generate_answer(question)
    print(f"\n💬 Answer: {answer}\n")
        
    # Judge answer
    print("⚖️  Judging answer...")
    evaluation = judge_answer(question, answer)
    print_evaluation(evaluation)
    
    print("\n" + "=" * 70)


PHASE 2: STRUCTURED LLM-AS-JUDGE


EXAMPLE 1

❓ Question: What is machine learning?

🤖 Generating answer...

💬 Answer: Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit programming. Instead of relying on predefined rules, machine learning systems learn from data by identifying patterns and making predictions or decisions based on that data.

The process typically involves the following steps:

1. **Data Collection**: Gathering relevant data that the machine learning model will use for training. This data can be structured (like spreadsheets) or unstructured (like images and text).

2. **Preprocessing**: Cleaning and organizing the data to make it suitable for analysis. This may involve handling missing values, normalizing data, and encoding categorical variables.

3. **Model Selection**: Choosing an appropriate machine learning algorithm based on the p